In [2070]:
# Standard library imports
import os
import sys
import re
import warnings
import random
import hashlib

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning and preprocessing
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  # Assuming you might need it

# Specific models and tools
from xgboost import XGBClassifier
import xgboost as xgb

# Encoding and feature selection
from category_encoders import TargetEncoder  # Fixed the import based on usage
from scipy.stats import randint, uniform

# Model persistence
from joblib import dump, load

# Miscellaneous settings
%matplotlib inline
warnings.filterwarnings('ignore')


In [2071]:
competitions = [

    # Code, Seasons
    ['E0', [2324, 2223, 2122, 2021, 1920]],
    ['D1', [2324, 2223, 2122, 2021, 1920]],
    ['I1', [2324, 2223, 2122, 2021, 1920]],

]

In [2072]:
matches_files = []

In [2073]:
for comp in competitions:

    for season in comp[1]:

        matches_files.append(f"data/matches/{comp[0]}_{season}.csv")

In [2074]:
# Load and concatenate matches data into a single DataFrame
df = pd.DataFrame()

for file in matches_files:

    try:
        df_temp = pd.read_csv(file)
        df = pd.concat([df, df_temp], ignore_index=True)
    except:
        # print an error message
        print(f'Error: {file} not found')

# print the amount of data loaded
print(f"Data loaded: {df.shape[0]} matches")

Data loaded: 5080 matches


In [2075]:
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2076]:
# Convert 'Div' to a categorical type, a numeric representation of the division
df['Div'] = df['Div'].astype('category').cat.codes

In [2077]:
# Create a unique list of HomeTeam and AwayTeam names combined, and add an index to each team
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()

# Sort the teams alphabetically
teams.sort()

# Convert to an array of dictionaries
teams = [{'team': team, 'index': index} for index, team in enumerate(teams)]

In [2078]:
# Create a unique list of Referees, and add an index to each Referee
referees = pd.concat([df['Referee']]).unique()

# Convert to an array of dictionaries
referees = [{'referee': referee, 'index': index} for index, referee in enumerate(referees)]

In [2079]:
# Map the team names to the index values in the 'teams' list
df['Team_ID'] = df['HomeTeam'].map({team['team']: team['index'] for team in teams})
df['Opp_ID'] = df['AwayTeam'].map({team['team']: team['index'] for team in teams})

In [2080]:
# Correcting the creation of a unique list of Referees and adding an index to each Referee
referees = df['Referee'].unique()  # This should directly refer to the 'Referee' column

# Convert to a dictionary with referee names as keys and their indices as values
referee_dict = {referee: index for index, referee in enumerate(referees)}

# Now map the 'Referee' column to these indices
df['Ref_ID'] = df['Referee'].map(referee_dict)

In [2081]:
# Date is in DD/MM/YYYY format, convert it to a datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Declare Date_temp as a temporary column, an 8 digit integer representation of the date
df['Date_temp'] = df['Date'].dt.year * 10000 + df['Date'].dt.month * 100 + df['Date'].dt.day

# Parse Date_temp to an 8 digit integer
df['Date_temp'] = df['Date_temp'].astype(int)

# Connvert 'Time', which is now in HH:MM format to a 4 digit integer
# Assuming a default time of 00:00 for missing values
df['Time'] = df['Time'].fillna('00:00').str.replace(':', '').astype(int)

In [2082]:
# Create an array of all HomeTeam and AwayTeam names
teams = np.append(df['HomeTeam'].unique(), df['AwayTeam'].unique())

In [2083]:
df.columns = [re.sub(r'[<]', '_st_', str(col)) for col in df.columns]
df.columns = [re.sub(r'[>]', '_gt_', str(col)) for col in df.columns]

In [2084]:
# Sort df by Date_temp and Time
df = df.sort_values(['Date_temp', 'Time'])

In [2085]:
def history_vs_opponent_weighted(df, row, team_column):
    # Initialize the total weighted score
    weighted_score = 0
    opponent_column = 'Opp_ID'

    row_date_temp = row['Date'].year * 10000 + row['Date'].month * 100 + row['Date'].day

    
    # Filter the DataFrame for matches between the specified team and opponent from the same season, excluding the current match
    filtered_matches = df[(df[team_column] == row[team_column]) & 
                          (df[opponent_column] == row[opponent_column]) &
                          (df['Date_temp'] < row_date_temp)]
    
    recent_matches = filtered_matches.sort_values(by='Date', ascending=False).head(5)
    
    # Calculate weights - newer matches have higher weights
    weights = range(len(recent_matches), 0, -1)  # Descending list based on the number of matches
    
    # Calculate score based on the match result
    for match, weight in zip(recent_matches.itertuples(), weights):
        if getattr(match, 'FTR') == 'H' and getattr(match, team_column) == getattr(match, 'Team_ID') or \
           getattr(match, 'FTR') == 'A' and getattr(match, team_column) != getattr(match, 'Team_ID'):
            weighted_score += 3 * weight  # Team won
        elif getattr(match, 'FTR') == 'A':
            weighted_score += 1 * weight  # Draw
        
    # Normalize the weighted score by the sum of weights
    normalized_weighted_score = weighted_score / sum(weights) if weights else 0

    #print(f"Weighted score: {weighted_score}, Normalized weighted score: {normalized_weighted_score}")

    return normalized_weighted_score

In [2086]:
# Apply the modified function to create new columns
df['team_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Team_ID'), axis=1)
df['opp_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Opp_ID'), axis=1)

In [2087]:
import pandas as pd

# Function adapted for DataFrame application
def convert_odds(row):
    odds_win, odds_draw, odds_lose = row['AvgH'], row['AvgD'], row['AvgA']
    prob_win = 1 / odds_win
    prob_draw = 1 / odds_draw
    prob_lose = 1 / odds_lose
    prob_not_win = prob_draw + prob_lose
    return pd.Series([prob_win, prob_not_win], index=['probs_win', 'probs_not_win'])

# Apply the function and create new columns
df[['probs_win', 'probs_not_win']] = df.apply(convert_odds, axis=1)

#df = df.drop(columns=['AvgH', 'AvgD', 'AvgA'])

In [2088]:
# Drop multiple columns
#df = df.drop(['Date'], axis=1)

In [2089]:
df.tail(5)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbMxAHA,BbAvAHA,Team_ID,Opp_ID,Ref_ID,Date_temp,team_hist_vs,opp_hist_vs,probs_win,probs_not_win
3555,2,2024-04-01,1130,Bologna,Salernitana,3,0,H,2,0,...,NaN,NaN,8,59,37,20240401,1.000000,0.0,0.775194,0.272712
3556,2,2024-04-01,1400,Cagliari,Verona,1,1,D,0,1,...,NaN,NaN,14,73,37,20240401,0.833333,1.0,0.458716,0.589234
3557,2,2024-04-01,1400,Sassuolo,Udinese,1,1,D,1,1,...,NaN,NaN,61,70,37,20240401,0.500000,1.6,0.348432,0.699511
3558,2,2024-04-01,1700,Lecce,Roma,0,0,D,0,0,...,NaN,NaN,39,58,37,20240401,0.333333,1.8,0.230947,0.816006
3559,2,2024-04-01,1945,Inter,Empoli,2,0,H,1,0,...,NaN,NaN,36,22,37,20240401,1.666667,1.4,0.833333,0.217517


In [2090]:
def team_form(df, row, perspective):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Initialize points
    points = 0
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    
    # Calculate points with weights
    weighted_points_sum = 0
    total_weights = sum(weights[:len(past_matches)])  # Adjust the total weight in case of less than 5 matches
    
    for match, weight in zip(past_matches.itertuples(), weights):
        if (match.Team_ID == team_id and match.FTR == 'H') or (match.Opp_ID == team_id and match.FTR == 'A'):
            points += 3
        elif match.FTR == 'D':
            points += 1
        else:
            points += 0

        weighted_points_sum += points * weight
    
    if total_weights > 0:
        return weighted_points_sum / total_weights
    else:
        return 0  # Return 0 if no past matches found

In [2091]:
# Applying the function to each row for 'Team'
df['team_form_team'] = df.apply(lambda row: team_form(df, row, 'Team'), axis=1)

# Applying the function to each row for 'Opp'
#df['team_form_opp'] = df.apply(lambda row: team_form(df, row, 'Opp'), axis=1)

In [2092]:
def rolling_avgs(df, row, perspective, home_column, away_column):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past 5 matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    values = []
    
    # Determine which column to use and collect the values
    for match in past_matches.itertuples():
        if match.Team_ID == team_id:
            values.append(getattr(match, home_column))  # Use home_column for home team
        else:
            values.append(getattr(match, away_column))  # Use away_column for away team
    
    # Calculate the weighted average of the values
    weighted_sum = sum(value * weight for value, weight in zip(values, weights))
    total_weights = sum(weights[:len(values)])  # Adjust total weight if there are less than 5 matches
    
    if total_weights > 0:
        return weighted_sum / total_weights
    else:
        return 0  # Return 0 if no past matches found


In [2093]:
df['team_shots'] = df.apply(lambda row: rolling_avgs(df, row, 'Team', 'HS', 'AS'), axis=1)
df['opp_shots'] = df.apply(lambda row: rolling_avgs(df, row, 'Opp', 'HS', 'AS'), axis=1)

df['team_shots_target'] = df.apply(lambda row: rolling_avgs(df, row, 'Team', 'HST', 'AST'), axis=1)
df['opp_shots_target'] = df.apply(lambda row: rolling_avgs(df, row, 'Opp', 'HST', 'AST'), axis=1)

In [2094]:
from datetime import timedelta

def avg_games_played(df, row, team_column):
    team = row[team_column]
    # Ensure current_match_date is a Timestamp for comparison
    current_match_date = pd.to_datetime(row['Date'], dayfirst=True)  # Assuming 'Date' format is 'dd/mm/yy'

    delta = 30
    start_date = current_match_date - timedelta(days=delta)

    # Ensure 'Date' column is in datetime format for comparison
    #df['Date_temp'] = pd.to_datetime(df['Date'], dayfirst=True)  # Convert 'Date' column to datetime if not already done

    # Filter the DataFrame for matches within the last 30 days
    if team_column == 'Team_ID':
        past_matches = df[((df[team_column] == team) | (df['Opp_ID'] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] < current_match_date)]
    else:
        past_matches = df[((df['Team_ID'] == team) | (df[team_column] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] < current_match_date)]

    # If no matches were played in the last 30 days
    if past_matches.empty:
        return 0

    # Calculate weights based on the recency of each match
    weights = (current_match_date - past_matches['Date']).dt.days
    weighted_count = sum(delta - weights + 1)  # '+ 1' to include the match day in the weight

    # Normalize weights to sum to 1 and calculate the weighted average
    total_weight = sum(delta - weights + 1)
    weighted_avg = weighted_count / total_weight

    return weighted_avg


In [2095]:
# Apply the function for each team and opponent
#df['team_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Team_ID'), axis=1)
#df['opp_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Opp_ID'), axis=1)

In [2096]:
# Calculate means only for numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
means = df[numeric_cols].mean()

# Fill missing values in numeric columns with their respective means
df[numeric_cols] = df[numeric_cols].fillna(means)

In [2097]:
df['FTR'] = df['FTR'].map({'H': 1, 'D': 0, 'A': 0}).astype(int)

In [2098]:
df.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,Date_temp,team_hist_vs,opp_hist_vs,probs_win,probs_not_win,team_form_team,team_shots,opp_shots,team_shots_target,opp_shots_target
3555,2,2024-04-01,1130,Bologna,Salernitana,3,0,1,2,0,...,20240401,1.000000,0.0,0.775194,0.272712,5.000000,16.266667,13.133333,5.600000,3.933333
3556,2,2024-04-01,1400,Cagliari,Verona,1,1,0,0,1,...,20240401,0.833333,1.0,0.458716,0.589234,3.466667,9.533333,12.333333,3.200000,4.400000
3557,2,2024-04-01,1400,Sassuolo,Udinese,1,1,0,1,1,...,20240401,0.500000,1.6,0.348432,0.699511,2.000000,9.866667,13.133333,2.733333,3.000000
3558,2,2024-04-01,1700,Lecce,Roma,0,0,0,0,0,...,20240401,0.333333,1.8,0.230947,0.816006,3.400000,14.266667,10.066667,3.466667,4.600000
3559,2,2024-04-01,1945,Inter,Empoli,2,0,1,1,0,...,20240401,1.666667,1.4,0.833333,0.217517,5.000000,13.600000,10.066667,5.466667,2.533333


In [2099]:
df = df[[
    
        'Div', 'Date_temp', 'Time', 'Team_ID', 'Opp_ID', 'Ref_ID', 'FTR', 
        'team_hist_vs', 
        #'opp_hist_vs',

        #'probs_win',         
        #'probs_not_win', 
        
        #'team_form_team', 
        #'team_form_opp',
         
        #'team_shots', 'opp_shots',
        #'team_shots_target', 'opp_shots_target',

        #'team_avg_games', 'opp_avg_games',

        'AvgH', 'AvgD', 'AvgA'
         
         
         ]]

In [2124]:
# Sort the DataFrame by Date_temp and separate the 200 most recent matches into a validation set
df.sort_values('Date_temp', inplace=True)

# Drop the 'Date_temp' column
df.drop('Date_temp', axis=1, inplace=True)

df_val = df.tail(200)
df = df.iloc[:-200]

KeyError: 'Date_temp'

In [ ]:
#import train_test_split
from sklearn.model_selection import train_test_split

# Split the data into X and y
X = df.drop('FTR', axis=1)
y = df['FTR']

X.columns = [re.sub(r'[<]', '_st_', str(col)) for col in X.columns]
X.columns = [re.sub(r'[>]', '_gt_', str(col)) for col in X.columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train), len(X_test)

(3984, 996)

In [ ]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier


# Create a pipeline
pipeline = Pipeline([
    ('target_encoder', TargetEncoder()),
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier())
])

# Define the hyperparameters

param_distributions = {

    'target_encoder__smoothing': randint(1, 100),
    'xgb__n_estimators': randint(100, 1000),
    'xgb__max_depth': randint(3, 10),
    'xgb__learning_rate': uniform(0.01, 0.6),
    'xgb__subsample': uniform(0.3, 0.7),
    'xgb__colsample_bytree': uniform(0.3, 0.7),
    'xgb__gamma': randint(0, 5),
    'xgb__reg_alpha': uniform(0, 1),
    'xgb__reg_lambda': uniform(0, 1),
    'xgb__min_child_weight': randint(1, 10),
    'xgb__scale_pos_weight': uniform(0.5, 1.5)
  
}

# Create a RandomizedSearchCV object
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=5,
    cv=TimeSeriesSplit(n_splits=5),
    scoring='f1',
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit the model
search.fit(X_train, y_train)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Pipeline(steps=[('target_encoder',
                                              TargetEncoder()),
                                             ('scaler', StandardScaler()),
                                             ('xgb',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            device=None,
                                                            early_stopping_rounds=None...
                                        'xgb__reg_alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000023EC825C290>,
                                        'xgb__reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000023EC82E5D60>,
                                        'xgb__scale_pos_weight': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000023D8E8515B0>,
                                        'xgb__subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000023EC825F320>},
                   random_state=42, scoring='f1', verbose=1)

In [ ]:
# print the classification report
print(classification_report(y_test, search.predict(X_test)))

              precision    recall  f1-score   support

           0       0.64      0.62      0.63       531
           1       0.58      0.60      0.59       465

    accuracy                           0.61       996
   macro avg       0.61      0.61      0.61       996
weighted avg       0.61      0.61      0.61       996



In [ ]:
def generate_sliding_windows(X, window_size, step):
    n_samples = len(X)
    windows = []
    for start_idx in range(0, n_samples - window_size + 1, step):
        end_idx = start_idx + window_size
        if end_idx > n_samples:
            break  # Avoid going beyond the dataset
        train_indices = list(range(max(0, start_idx - window_size), start_idx))
        test_indices = list(range(start_idx, end_idx))
        windows.append((train_indices, test_indices))
    return windows

negative_count = len(df[df['FTR'] == 0])
positive_count = len(df[df['FTR'] == 1])
scale_pos_weight_value = negative_count / positive_count

# Define the hyperparameter search space
param_dist = {
    
    'xgb__clf__max_depth': [1,2,3],
    'xgb__clf__learning_rate': [0.001, 0.01, 0.1],
    'xgb__clf__lambda': [1, 1.5, 2],  # L2 regularization term on weights
    'xgb__clf__alpha': [0, 0.5, 1],  # L1 regularization term on weights
    'xgb__clf__n_estimators': [1, 5, 100],

    'rf__clf__max_depth': [None, 4, 6],
    'rf__clf__min_samples_split': [2, 5],
    'rf__clf__min_samples_leaf': [1, 2],
    'rf__clf__bootstrap': [True, False],
    'rf__clf__n_estimators': [50, 100, 200],

    'lr__clf__C': [0.1, 1, 10],  # Inverse of regularization strength; smaller values specify stronger regularization.
    'lr__clf__penalty': ['l1', 'l2', 'elasticnet'],  # Specify the norm of the penalty.
    'lr__clf__solver': ['saga'],  # Algorithm to use in the optimization problem, 'saga' supports all penalties.
    'lr__clf__l1_ratio': [0.5],  # The Elastic-Net mixing parameter, with 0 <= l1_ratio <= 1. Only used if penalty='elasticnet'.

    'cat__clf__depth': [1,2,3,4],
    'cat__clf__learning_rate': [0.01, 0.05, 0.1],
    'cat__clf__iterations': [50, 100, 200],
    'cat__clf__l2_leaf_reg': [1, 3, 5],

    'gb__clf__learning_rate': [0.01, 0.1, 0.2],
    'gb__clf__n_estimators': [50, 100, 200],
    'gb__clf__max_depth': [3, 5, 7],
    'gb__clf__min_samples_split': [2, 5],
    'gb__clf__min_samples_leaf': [1, 2],

}


param_test = {
    
    'xgb__clf__max_depth': [1,2,3],
    'xgb__clf__learning_rate': [0.001, 0.01, 0.1],
    'xgb__clf__lambda': [1, 1.5, 2],  # L2 regularization term on weights
    'xgb__clf__alpha': [0, 0.5, 1],  # L1 regularization term on weights
    'xgb__clf__n_estimators': [1, 5, 100],


}

In [ ]:
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Define a custom scoring function
def xgb_early_stopping_score(y, estimator, X, y_true, sample_weight=None):
    """
    Custom scorer that uses early stopping.
    """
    # Split X into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)
    
    # Fit with early stopping
    eval_set = [(X_val, y_val)]
    estimator.fit(X_train, y_train, early_stopping_rounds=10, eval_set=eval_set, verbose=False)
    
    # Predict on the validation set
    y_pred = estimator.predict(X_val)
    
    # Return the F1 score
    return f1_score(y_val, y_pred, pos_label=1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import make_scorer, f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from imblearn.pipeline import Pipeline as ImbPipeline
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# LightGBM
from lightgbm import LGBMClassifier

# naive bayes
from sklearn.naive_bayes import GaussianNB

#catboost
from catboost import CatBoostClassifier

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Define the F1 score for the '1' class
f1_scorer = make_scorer(f1_score, pos_label=1)

best_f1_score = 0
best_f1_params = None
best_window_size = None
best_precision = 0
best_model = None 
f1_scores = []
precision_scores = []

# Define your dataset
# Assuming 'df' is your DataFrame
#X = train_set.drop(columns=['result'])
#y = train_set['result']

# Make the custom scorer
custom_scorer = make_scorer(xgb_early_stopping_score, greater_is_better=True, needs_proba=False, X=X, y_true=y)

# Set the window_size and step to 5% of the dataset
window_size = int(len(X) * 0.2)
step = int(len(X) * 0.2)

# Initialize an empty list to store precision scores
precision_scores = []

# Initialize an empty dataframe to store misclassified samples
misclassified_samples = pd.DataFrame(columns=X.columns)

# Generate windows
window_splits = generate_sliding_windows(X, window_size, step)

# Initialize training indices with the first window
train_end_index = window_size

# Iterate over each sliding window
for i, (train_index, test_index) in enumerate(window_splits):

    # Update training indices to include the next window
    train_index = list(range(train_end_index))
    train_end_index += window_size

    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]

    print(f"Iteration {i+1} Training Data Shape: {X_train.shape}")

    # Combine misclassified samples from previous iterations with current training data
    if not misclassified_samples.empty:
        X_train_combined = pd.concat([X_train, misclassified_samples[X_train.columns]], axis=0)
        y_train_combined = pd.concat([y_train, misclassified_samples['FTR']], axis=0)
    else:
        X_train_combined = X_train
        y_train_combined = y_train

    # Calculate misclassification frequency
    misclassified_freq = y_train_combined.value_counts(normalize=True)

    # Define class weights based on misclassification frequency
    class_weights = {0: 1, 1: max(0.6, 1 - misclassified_freq.get(1, 0.5))}  # Adjust dynamically to penalize misclassification of class 1 more heavily

    # Define pipelines for each classifier with SMOTE and TargetEncoder
    pipeline_xgb = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', XGBClassifier(random_state=42, verbose=0))
    ])

    pipeline_gb = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', GradientBoostingClassifier(random_state=42, verbose=0))
    ])

    # pipeline for logistic regression
    pipeline_lr = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', LogisticRegression(random_state=42, verbose=0))
    ])

    # pipeline for catboost classifier
    pipeline_cat = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', CatBoostClassifier(random_state=42, verbose=0))
    ])

    # pipeline for random forest
    pipeline_rf = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', RandomForestClassifier(random_state=42, verbose=0))
    ])

    # LightGBM pipeline
    pipeline_lgbm = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', LGBMClassifier(random_state=42, force_col_wise='true', verbose=0))
    ])

    # Adaboost pipeline
    pipeline_ada = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', AdaBoostClassifier(random_state=42))
    ])

    # Combine them into an ensemble classifier
    ensemble_clf = VotingClassifier(estimators=[
        ('xgb', pipeline_xgb),
        #('gb', pipeline_gb),
        ('lr', pipeline_lr),
        #('cat', pipeline_cat),
        #
        #('rf', pipeline_rf),
        #('lgbm', pipeline_lgbm),
        ('ada', pipeline_ada)
    ], voting='soft')

    # Setup RandomizedSearchCV
    clf = RandomizedSearchCV(
        estimator=ensemble_clf,
        param_distributions=param_test,
        n_iter=5,
        scoring=f1_scorer,
        cv=TimeSeriesSplit(n_splits=3),
        random_state=42,
        n_jobs=-1,
        verbose=0
    )  

    # Fit RandomizedSearchCV
    clf.fit(X_train_combined, y_train_combined)

    # Get the best parameters
    best_params = clf.best_params_
    print("Best Parameters:", best_params)

    # Use the best estimator
    best_pipe = clf.best_estimator_

    # Make predictions
    y_proba = best_pipe.predict_proba(X_test)

    # Apply threshold
    threshold = 0.5  # You can adjust this threshold as needed
    y_pred = (y_proba[:, 1] >= threshold).astype(int)

    current_f1_score = f1_score(y_test, y_pred, pos_label=1)

    if current_f1_score > best_f1_score:
        best_f1_score = current_f1_score
        best_f1_params = clf.best_params_
        #best_model = clf.best_estimator_ 

    # ------------------------------------------------

    best_model = clf.best_estimator_ 

    # Calculate precision score
    precision = np.mean(y_test == y_pred)
    precision_scores.append(precision)
    print("Precision:", precision)

    print()

# Print the best F1 score and its corresponding parameters
print()
print("Best F1 Score:", best_f1_score)

# print the classification report of the best model on the full dataset
print(classification_report(y_test, y_pred))

Iteration 1 Training Data Shape: (996, 9)


Best Parameters: {'xgb__clf__n_estimators': 100, 'xgb__clf__max_depth': 3, 'xgb__clf__learning_rate': 0.01, 'xgb__clf__lambda': 2, 'xgb__clf__alpha': 0.5}
Precision: 0.6455823293172691

Iteration 2 Training Data Shape: (1992, 9)
Best Parameters: {'xgb__clf__n_estimators': 5, 'xgb__clf__max_depth': 2, 'xgb__clf__learning_rate': 0.1, 'xgb__clf__lambda': 2, 'xgb__clf__alpha': 1}
Precision: 0.6937751004016064

Iteration 3 Training Data Shape: (2988, 9)
Best Parameters: {'xgb__clf__n_estimators': 5, 'xgb__clf__max_depth': 2, 'xgb__clf__learning_rate': 0.1, 'xgb__clf__lambda': 2, 'xgb__clf__alpha': 1}
Precision: 0.6646586345381527

Iteration 4 Training Data Shape: (3984, 9)
Best Parameters: {'xgb__clf__n_estimators': 100, 'xgb__clf__max_depth': 3, 'xgb__clf__learning_rate': 0.01, 'xgb__clf__lambda': 2, 'xgb__clf__alpha': 0.5}
Precision: 0.6616465863453815

Iteration 5 Training Data Shape: (4980, 9)
Best Parameters: {'xgb__clf__n_estimators': 100, 'xgb__clf__max_depth': 3, 'xgb__clf__learning

In [ ]:
# Correlation matrix
corr = df.corr()

# Put the target column to the front
cols = list(corr.columns)
cols.insert(0, cols.pop(cols.index('FTR')))
corr = corr.loc[cols, cols]

# Plot the correlation matrix
#plt.figure(figsize=(10, 8))
#sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=2)
#plt.show()



In [ ]:
# Print the feature importances
importances = search.best_estimator_.named_steps['xgb'].feature_importances_
features = X_train.columns
importances_df = pd.DataFrame({'features': features, 'importances': importances})
importances_df = importances_df.sort_values('importances', ascending=False)

# Plot the feature importances
#plt.figure(figsize=(10, 8))
#sns.barplot(x='importances', y='features', data=importances_df)
#plt.title('Feature Importances')
#plt.show()



### Validation

In [2162]:
threshold = 0.589

# Apply the best estimator to the validation set
y_val_proba = best_model.predict_proba(df_val.drop(columns=['FTR']))
y_val_pred = (y_val_proba[:, 1] >= threshold).astype(int)

# Print the classification report
print(classification_report(df_val['FTR'], y_val_pred))

              precision    recall  f1-score   support

           0       0.70      0.97      0.81        60
           1       0.88      0.38      0.53        40

    accuracy                           0.73       100
   macro avg       0.79      0.67      0.67       100
weighted avg       0.77      0.73      0.70       100



In [2163]:
# Display the predictions and actual results of the validation set side by side
predictions = pd.DataFrame({'Prediction': y_val_pred, 'Actual': df_val['FTR']})
print(predictions)



      Prediction  Actual
3514           0       0
256            1       1
3513           1       1
2007           0       0
3517           0       1
...          ...     ...
3558           0       0
3555           1       1
3556           0       0
3557           0       0
3559           1       1

[100 rows x 2 columns]


In [ ]:
# Apply the model to the validation set
y_val = df_val['FTR']
X_val = df_val.drop('FTR', axis=1)
y_pred = search.predict(X_val)

# print the classification report
print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.71      0.50      0.59        60
           1       0.48      0.70      0.57        40

    accuracy                           0.58       100
   macro avg       0.60      0.60      0.58       100
weighted avg       0.62      0.58      0.58       100

